In [1]:
%pip install gurobipy -qU

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
# A first dataset, used in the slides
nr_items = 3
items = range(0,nr_items)
preferences = [[0,0.2,0],[0.8,0,0.2],[1,0.8,0]]
orders = [[[0,1,0],[0,0,0],[1,1,0]],[[0,0,1],[1,0,1],[0,0,0]]]

In [4]:
# Create master model
master = gp.Model("MasterRUM")
master.setParam('OutputFlag', 0)

# Create initial variables. We have 1 initial variable per included preference order.
x = {}
for i in range(0,len(orders)):
    x[i] = master.addVar(name = "x[%d]" %i)
    
# Create a slack variable, showing how far from a feasible solution we are.
# We immediately include the objective coefficient
slack = master.addVar(obj = 1, name = "slack")

# Constraints to enforce preferences are explained
constraints = {}
for i in items:
    constraints[i] = master.addConstrs(gp.quicksum(orders[k][i][j]*x[k] for k in range(0,len(orders))) + slack >= preferences[i][j] for j in items)     

# Constraints to enforce sum of preference orders does not exceed 1
# Since the objective is minimize, we multiply the natural constraint by -1 to keep greater than constraints.
convex_const = master.addConstr(gp.quicksum(-x[k] for k in range(0,len(orders))) >= -1)

Restricted license - for non-production use only - expires 2025-11-24


In [5]:
# Create pricing model
pricing = gp.Model("PricingRUM")
pricing.setParam('OutputFlag', 0)

# Create pricing variables
y = {}
for i in items:
    y[i] = {}
    for j in items:
        if i != j:
            y[i][j] = pricing.addVar(vtype = GRB.INTEGER, name = "y[%d][%d]"%(i,j))
            
# Constraints to enforce strict preferences. Between two alternatives, one must be preferred over the other.
for i in items:
    for j in range(i+1,nr_items):
        pricing.addConstr(y[i][j] + y[j][i] == 1)
            
# Constraints to enforce transitivity
for i in items:
    for j in range(i+1,nr_items):
        for k in range(j+1,nr_items):
            pricing.addConstr(y[i][j] + y[j][k] + y[k][i] <= 2)
            pricing.addConstr(y[i][k] + y[k][j] + y[j][i] <= 2)


In [6]:
def printMasterSolution():
    if master.status == GRB.OPTIMAL:
        print("Master Solution value = %f" %master.Objval)
        print("Slack = %f" %slack.X)
        for i in x:
            print("Variable x[%d] = %f" %(i,x[i].X))
    else:
        print('No solution')
        
def printPricingProblem():
    pricing.write("RUMPricing.lp")
    f = open("RUMPricing.lp","r")
    contents = f.read()
    print(contents)

In [7]:
while 1:    
    master.optimize()
    printMasterSolution()
    
    pricing.setObjective(gp.quicksum(constraints[i][j].Pi*y[i][j] for i in items for j in items if i != j), GRB.MAXIMIZE)
    printPricingProblem()
    pricing.optimize()
    # If a new preference order is found, we add it to the master problem.
    if pricing.ObjVal > convex_const.Pi:
        improve = 1
        new_order = []
        for i in items:
            new_line = []
            for j in items:
                if i != j:
                    new_line.append(y[i][j].X)
                else:
                    new_line.append(0)
            new_order.append(new_line)
        # Due to numerical issues, it is possible that patterns incorrectly get identified as improving. If this happens, the same
        # patterns will keep being added, and not being used in the master problem. The following statement is to identify this
        # situation if it happens.
        if new_order == orders[len(orders)-1]:
            break
        orders.append(new_order)
        print(new_order)
        # Build the new variable (and include in the objective with coefficient 1)
        x[len(orders)-1] = master.addVar()
        # This includes the new variable/pattern in the constraints.
        for i in items:
            for j in items:
                master.chgCoeff(constraints[i][j], x[len(orders)-1], new_order[i][j])
        master.chgCoeff(convex_const, x[len(orders)-1], -1)
    else:
        break

Master Solution value = 0.400000
Slack = 0.400000
Variable x[0] = 0.600000
Variable x[1] = 0.400000
\ Model PricingRUM
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  0.5 y[1][0] + 0.5 y[2][0]
Subject To
 R0: y[0][1] + y[1][0] = 1
 R1: y[0][2] + y[2][0] = 1
 R2: y[1][2] + y[2][1] = 1
 R3: y[0][1] + y[1][2] + y[2][0] <= 2
 R4: y[0][2] + y[1][0] + y[2][1] <= 2
Bounds
Generals
 y[0][1] y[0][2] y[1][0] y[1][2] y[2][0] y[2][1]
End

[[0, -0.0, -0.0], [1.0, 0, 1.0], [1.0, 0.0, 0]]
Master Solution value = 0.300000
Slack = 0.300000
Variable x[0] = 0.500000
Variable x[1] = 0.000000
Variable x[2] = 0.500000
\ Model PricingRUM
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  0.5 y[1][0] + 0.5 y[2][1]
Subject To
 R0: y[0][1] + y[1][0] = 1
 R1: y[0][2] + y[2][0] = 1
 R2: y[1][2] + y[2][1] = 1
 R3: y[0][1] + y[1][2] + y[2][0] <= 2
 R4: y[0][2] + y[1][0] + y[2][1] <= 2
Bounds
Generals
 y[0][1] y[0][2] y[1][0] y[1][2] y[

In [8]:
master.write("RUMMaster.lp")
f = open("RUMMaster.lp","r")
contents = f.read()
print(contents)

\ Model MasterRUM
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  slack
Subject To
 R0: slack >= 0
 R1: x[0] + slack >= 0.2
 R2: x[1] + slack >= 0
 R3: x[1] + slack + C3 + C4 >= 0.8
 R4: slack >= 0
 R5: x[1] + slack + C3 >= 0.2
 R6: x[0] + slack + C3 + C4 >= 1
 R7: x[0] + slack + C4 >= 0.8
 R8: slack >= 0
 R9: - x[0] - x[1] - C3 - C4 >= -1
Bounds
End



In [9]:
pricing.write("RUMPricing.lp")
f = open("RUMPricing.lp","r")
contents = f.read()
print(contents)

\ Model PricingRUM
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  0.5 y[0][1] + 0.5 y[1][0]
Subject To
 R0: y[0][1] + y[1][0] = 1
 R1: y[0][2] + y[2][0] = 1
 R2: y[1][2] + y[2][1] = 1
 R3: y[0][1] + y[1][2] + y[2][0] <= 2
 R4: y[0][2] + y[1][0] + y[2][1] <= 2
Bounds
Generals
 y[0][1] y[0][2] y[1][0] y[1][2] y[2][0] y[2][1]
End

